## LLM zoomcamp Homework 1

### Task 1 

Run Elastic Search 8.17.6, and get the cluster information. If you run it on localhost, this is how you do it:

docker stop elasticsearch
docker rm elasticsearch
docker pull docker.elastic.co/elasticsearch/elasticsearch:8.17.6

docker run -d --name elasticsearch -p 9200:9200 -p 9300:9300 -e "discovery.type=single-node" -e "ELASTIC_PASSWORD=123456" docker.elastic.co/elasticsearch/elasticsearch:8.17.6

docker start elasticsearch
docker exec -it elasticsearch bin/elasticsearch-reset-password -u elastic

In [1]:
from elasticsearch import Elasticsearch
import os 
from dotenv import load_dotenv

In [7]:
load_dotenv()

ELASTIC_PASSWORD = os.getenv('ELASTIC_PASSWORD')

In [3]:
!curl http://localhost:9200

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
curl: (52) Empty reply from server


In [4]:
es_client = Elasticsearch('http://localhost:9200') 


In [5]:
! curl http://localhost:9200

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
curl: (52) Empty reply from server
